In [ ]:
# to be deleted
from deep_translator import GoogleTranslator

def translate_text(text):
    translator = GoogleTranslator(source='vi', target='en')
    try:
        trans_text = translator.translate(text)
        return trans_text

    except Exception as e:
        return "Error: " + str(e)

# Use this to map translating only for test hf dataset
def translate_text_google(example):
    question = example['question']
    op1 = example['option_1']
    op2 = example['option_2']
    op3 = example['option_3']
    op4 = example['option_4']

    return {
        'trans_question': translate_text(question),
        'trans_option_1': translate_text(op1),
        'trans_option_2': translate_text(op2),
        'trans_option_3': translate_text(op3),
        'trans_option_4': translate_text(op4)
    }

In [ ]:
# to be deleted
from transformers import pipeline

def summarize_text(text,min_length=220, max_length=400):
    summarizer = pipeline("summarization", model='sshleifer/distilbart-cnn-12-6')
    summary = summarizer(text, max_length=max_length, min_length=min_length)
    return summary

In [ ]:
# to be deleted

def get_context_from_question_faiss(question,k,faiss_index_path,trans_corpus_path):
    model_name = 'sentence-transformers/all-mpnet-base-v2'
    model = SentenceTransformer(model_name)

    index = faiss.read_index(faiss_index_path)
    trans_corpus = pd.read_csv(trans_corpus_path)

    xq = model.encode(question)
    xq = np.expand_dims(xq, axis=0)
    D, I = index.search(xq, k)

    return [{trans_corpus['Url'][i]: trans_corpus['Content'][i]} for i in I[0]]

In [ ]:
# demo, to be deleted
question = 'What high technology is used to treat hormonal acne?'
get_context_from_question_faiss(question, k=2,faiss_index_path = '/content/drive/MyDrive/faiss_768.index',
                                                trans_corpus_path = '/content/drive/MyDrive/df_768.csv')

In [ ]:
# to be deleted
def dataset_get_context(example):
    id = example['id']
    level = id.split('_')[0]
    question = example['trans_ner_question']

    if level == 'level1':
        k=1
    else:
        k=2

    context = get_context_from_question_faiss(question=question, k=k,
                                                    faiss_index_path = '/content/drive/MyDrive/faiss_768.index',
                                                    trans_corpus_path = '/content/drive/MyDrive/df_768.csv')
    concat_context = "\n".join(context) # Concat all text in to long
    example['context'] = concat_context
    return example

In [ ]:
# use this for batch faiss encode

faiss_index_path = '/content/drive/MyDrive/faiss_768.index'
trans_corpus_path = '/content/drive/MyDrive/df_768.csv'
index = faiss.read_index(faiss_index_path)
trans_corpus = pd.read_csv(trans_corpus_path)

def dataset_get_context_batch(examples):
    id_batch = examples['id']
    level_batch = [id.split('_')[0] for id in id_batch]
    question_batch = examples['trans_ner_question']

    ## batch sentenceTransformers
    model_name = 'sentence-transformers/all-mpnet-base-v2'
    model = SentenceTransformer(model_name)

    xq_batch = model.encode(question_batch)
    context_batch = []
    for xq, level in zip(xq_batch, level_batch):
        if level == 'level1':
            k=1
        else:
            k=2
        xq = np.expand_dims(xq, axis=0)  # query vector
        D, I = index.search(xq, k)   # D: distance, I: index
        context = [trans_corpus['Content'][i] for i in I[0]]
        concat_context = "\n".join(context) # Concat all text in to long text

        context_batch.append(concat_context)

    examples['context'] = context_batch
    return examples# MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7  --- 270M params
# MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli   --- 435M params

classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", device=0, use_fast=True)

def get_binary_answer(choices: list, answers: list):
    length = len(choices)
    result = ['0']*length
    for answer in answers:
        for i in range(len(choices)):
            if answer == choices[i]:
                result[i] = '1'
    return ''.join(result)


def get_prediction(example):
    prompt = example['prompt']

    id = example['id']
    choices = [example['trans_option_1'], example['trans_option_2'],
               example['trans_option_3'], example['trans_option_4'], example['trans_option_5']]

    choices = [item for item in choices if item != 'Cloudy sky']

    predicted_labels = []
    threshold = 0.85

    results = classifier(prompt, choices, multi_label=True)
    scores = results['scores']
    labels = results['labels']

    highest_score_index = np.argmax(scores)
    my_answers = [labels[highest_score_index]]

    # Include labels with scores above the threshold
    for i in range(len(labels)):
        if (scores[i] >= threshold) and (labels[i] not in my_answers):
            my_answers.append(labels[i])

    binary_answer = get_binary_answer(choices, my_answers)
    binary_answer = f'{id},{binary_answer}'
    example['binary_answer'] = binary_answer

    return example

In [ ]:
# Get prediction using multiple context
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", device=0, use_fast=True)

def get_binary_answer(choices: list, answers: list):
    length = len(choices)
    result = ['0']*length
    for answer in answers:
        for i in range(len(choices)):
            if answer == choices[i]:
                result[i] = '1'
    return ''.join(result)


def get_multi_context_prediction(example):
    prompt_1 = example['prompt_1']
    prompt_2 = example['prompt_2']
    id = example['id']
    choices = [example['trans_option_1'], example['trans_option_2'],
               example['trans_option_3'], example['trans_option_4'], example['trans_option_5']]

    choices = [item for item in choices if item != 'Cloudy sky']

    threshold = 0.85

    results = classifier([prompt_1, prompt_2], choices, multi_label=True)
    scores_1, labels_1 = results[0]['scores'], results[0]['labels']
    scores_2, labels_2 = results[1]['scores'], results[1]['labels']

    max_score_1_index = np.argmax(scores_1)
    my_answers_1 = [labels_1[max_score_1_index]]

    max_score_2_index = np.argmax(scores_2)
    my_answers_2 = [labels_2[max_score_2_index]]

    # Include labels with scores above the threshold
    for i in range(len(choices)):
        if (scores_1[i] >= threshold) and (labels_1[i] not in my_answers_1):
            my_answers_1.append(labels_1[i])

        if (scores_2[i] >= threshold) and (labels_2[i] not in my_answers_2):
            my_answers_2.append(labels_2[i])

    binary_answer_1 = get_binary_answer(choices, my_answers_1)
    binary_answer_2 = get_binary_answer(choices, my_answers_2)

    binary_answer_1 = f'{id},{binary_answer_1}'
    binary_answer_2 = f'{id},{binary_answer_2}'
    example['binary_answer_1'] = binary_answer_1
    example['binary_answer_2'] = binary_answer_2
    return example

In [ ]:
# to be deleted when done
import datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def translate_term_for_mapping_batch(examples):
    ########### model AI translate
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model_name = '/content/drive/MyDrive/Model_Translation_envit5'
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

    term_batch = examples['Name']
    term_batch_prepend = [f'vi : {term}' for term in term_batch]

    inputs = tokenizer(term_batch_prepend, return_tensors="pt", padding="longest", truncation=True, max_length=56)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.inference_mode():
        outputs = model.generate(**inputs, min_length=0,max_length=56,no_repeat_ngram_size=2)

    translated_terms = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    trans_terms = [term.replace('en:','').replace('vi:','').strip() for term in translated_terms]

    examples['model_trans_term']= trans_terms

    ########### Google Translate
    ggtrans_terms = []
    for term in term_batch:
        ggtrans_term = translate_text(term)
        ggtrans_terms.append(ggtrans_term)

    examples['gg_trans_term']= ggtrans_terms

    return examples

In [ ]:
def predict_example(example):
    question = example['question']
    context = example['context']
    opa = example['trans_option_1']
    opb = example['trans_option_2']
    opc = example['trans_option_3']
    opd = example['trans_option_4']

    if opd != '':
        n_choice = 4
    elif opc != '':
        n_choice = 3
    elif opb != '':
        n_choice = 2

    options = [opa, opb, opc, opd][:n_choice]
    input = [f'question: {question}, answer: {option}, context: {context}' for option in [opa, opb, opc, opd][:n_choice]]

    tokenized = tokenizer(input, max_length=512, padding='max_length', truncation=True, return_tensors='pt')
    input_ids = tokenized['input_ids'].unsqueeze(dim=0)
    attention_mask = tokenized['attention_mask'].unsqueeze(dim=0)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    predicted_labels = torch.argmax(outputs.logits, dim=1)
    example['predicted_labels'] = predicted_labels.tolist()

    return example

prediction = test_ggtrans_context.map(predict_example)

In [ ]:
# Hard vote
def get_binary_answer(choices: list, answers: list):
    length = len(choices)
    result = ['0']*length
    for answer in answers:
        for i in range(len(choices)):
            if answer == choices[i]:
                result[i] = '1'
    return ''.join(result)

def ensemble_binary(scores: list, vote=2):
    length = len(scores[0])
    answer = ['0']*length
    for i in range(len(scores[0])):
        count = 0
        for j in range(len(scores)):
          if scores[j][i] == '1':
            count += 1
          if count == vote:
            answer[i] = '1'
    return ''.join(answer)

def final_ensemble(scores):
    length = len(scores[0])
    answer = ['0']*length
    for i in range(len(scores[0])):
        for j in range(len(scores)):
          if scores[j][i] == '1':
            answer[i] = '1'
            break
    return ''.join(answer)

def get_multi_context_prediction_ensemble_hard_vote(example):
    prompt_1 = example['prompt_1']
    prompt_2 = example['prompt_2']
    id = example['id']
    choices = [example['trans_option_1'], example['trans_option_2'],
               example['trans_option_3'], example['trans_option_4'], example['trans_option_5']]

    choices = [item for item in choices if item != 'cloudy sky']

    threshold = 0.87
    argmax_threshold = 0.67
    context1_for_ensemble = [] # result will be as binary : ['1001', '1001','1000',.....,'0001'] then used to merge into final answer
    context2_for_ensemble = []

    for classifier in classifiers:
        results = classifier([prompt_1, prompt_2], choices, multi_label=True)
        context1_scores, context1_labels = results[0]['scores'], results[0]['labels']
        context2_scores, context2_labels = results[1]['scores'], results[1]['labels']

        candidate1_answer = []
        candidate2_answer = []
        if np.max(context1_scores) >= argmax_threshold: # make sure max score > threshold
            max_score_1_index = np.argmax(context1_scores)
            candidate1_answer.append(context1_labels[max_score_1_index])

        if np.max(context2_scores) >= argmax_threshold: # make sure max score > threshold
            max_score_2_index = np.argmax(context2_scores)
            candidate2_answer.append(context2_labels[max_score_2_index])

        # Include labels with scores above the threshold
        for i in range(len(choices)):
            if (context1_scores[i] >= threshold) and (context1_labels[i] not in candidate1_answer):
                candidate1_answer.append(context1_labels[i])

            if (context2_scores[i] >= threshold) and (context2_labels[i] not in candidate2_answer):
                candidate2_answer.append(context2_labels[i])

        bin_candidate1_answer = get_binary_answer(choices, candidate1_answer)
        bin_candidate2_answer = get_binary_answer(choices, candidate2_answer)

        context1_for_ensemble.append(bin_candidate1_answer)
        context2_for_ensemble.append(bin_candidate2_answer)

    # Begin to ensemble answer
    vote = 3
    context1_final_answer = ensemble_binary(context1_for_ensemble, vote=vote)
    context2_final_answer = ensemble_binary(context2_for_ensemble, vote=vote)

    # Final ensemble
    final_answer = final_ensemble([context1_final_answer, context2_final_answer])
    binary_final_answer = f'{id},{final_answer}'

    example['context1_for_ensemble'] = context1_for_ensemble
    example['context2_for_ensemble'] = context2_for_ensemble
    example['context1_answer'] = context1_final_answer
    example['context2_answer'] = context2_final_answer
    example['binary_final_answer'] = binary_final_answer

    return example